In [ ]:
## Remote Python Runtime

In [80]:
import os
import re
from typing import List
from pathlib import Path
from langchain.docstore.document import Document
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import MarkdownHeaderTextSplitter

In [81]:
def hierarchical_markdown_split(md_text: str, path_prefix: str = "") -> list[Document]:
    """마크다운 문서를 계층적으로 분할합니다."""
    splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[
        ("#", "title"),
        ("##", "section"),
        ("###", "subsection"),
        ("####", "subsubsection"),
        ("#####", "subsubsubsection")
    ])
    docs = splitter.split_text(md_text)

    result_docs = []
    current_title = None
    chunk_idx = 0
    for doc in docs:
        metadata = doc.metadata
        if "title" in metadata:
            current_title = metadata["title"]

        if current_title:
            chunk_idx += 1
            full_title = ""
            if "subsection" in metadata:
                full_title += f" > {metadata['subsection']}"
            if "subsubsection" in metadata:
                full_title += f" > {metadata['subsubsection']}"
            if "subsubsubsection" in metadata:
                full_title += f" > {metadata['subsubsubsection']}"

            content = f"[{full_title}]\n\n{doc.page_content}"
            doc = Document(page_content=content, metadata={
                **doc.metadata,
                "type": "documentation",
                "source": "dev_center_guide_touched.md",
                "chunk_idx": chunk_idx
            })

        result_docs.append(doc)

    return result_docs

def load_markdown_file(file_path: str) -> str:
    """마크다운 파일을 로드합니다."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# 마크다운 파일 로드 및 분할
str_md_file = load_markdown_file("data/dev_center_guide_touched.md")
docs_markdown = hierarchical_markdown_split(str_md_file)
print(f"마크다운 문서 분할 완료: {len(docs_markdown)}개 청크")

마크다운 문서 분할 완료: 119개 청크


In [ ]:
def chunk_based_markdown_split(md_text: str, chunk_size: int = 500, chunk_overlap: int = 100) -> list[Document]:
    """
    마크다운 문서를 chunk_size와 chunk_overlap 기반으로 분할합니다.
    
    Args:
        md_text: 마크다운 텍스트
        chunk_size: 청크 크기 (기본값: 500)
        chunk_overlap: 청크 간 겹치는 문자 수 (기본값: 100)
        
    Returns:
        list[Document]: 분할된 문서 리스트
    """
    from langchain_text_splitters import RecursiveCharacterTextSplitter
    
    # RecursiveCharacterTextSplitter 사용
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ". ", " ", ""],  # 마크다운에 적합한 구분자
        length_function=len,
    )
    
    docs = splitter.split_text(md_text)
    
    # Document 객체로 변환하고 메타데이터 추가
    result_docs = []
    for i, doc_text in enumerate(docs):
        # 헤더 정보 추출 (첫 번째 줄에서 # 확인)
        lines = doc_text.split('\n')
        header_info = ""
        content_start = 0
        
        for j, line in enumerate(lines):
            if line.strip().startswith('#'):
                header_info = line.strip()
                content_start = j + 1
                break
        
        # 헤더 정보를 포함한 내용 구성
        if header_info:
            content = f"[{header_info}]\n\n" + '\n'.join(lines[content_start:])
        else:
            content = doc_text
        
        doc = Document(
            page_content=content,
            metadata={
                "type": "documentation",
                "source": "dev_center_guide_touched.md",
                "chunk_idx": i,
                "chunk_size": chunk_size,
                "chunk_overlap": chunk_overlap,
                "header": header_info if header_info else "no_header"
            }
        )
        result_docs.append(doc)
    
    return result_docs

In [82]:
# docs_markdown에서 'launchPurchaseFlow()' 문자열을 포함하는 Documnet 를 출력

cnt = 0
for doc in docs_markdown:
    if 'launchPurchaseFlow()' in doc.page_content:
        print(f"문서 제목: {doc.metadata.get('title', '제목 없음')}")
        print(f"문서 내용: {doc.page_content}...")  # 처음 200자만 출력
        print("-" * 40)  # 구분선 
        cnt += 1
        
print(f"'launchPurchaseFlow()' 문자열을 포함하는 문서 개수: {cnt}")
        

문서 제목: 원스토어 인앱 결제 연동 가이드
문서 내용: [ > 01. 원스토어 인앱결제 개요 > 결제 프로세스 > 인앱 상품 구매 요청하기]

상품을 구매하려면 먼저 상품 상세정보(queryProductDetailAsync) API 호출하여 상품의 정보을 가지고 launchPurchaseFlow API를 통해 구매 화면을 호출합니다.
결제가 완료되면 PurchaseClient 객체를 초기화할 때 입력한 PurchasesUpdatedListener를 통해 구매정보를 전달받을 수 있습니다.
결과 코드값을 검증한 후에 성공으로 확인되면 인앱 상품에 해당하는 아이템을 지급하는 프로세스를 진행합니다.
아래 그림은 개발사 앱과 원스토어 사이의 인앱 상품을 구매하는 프로세스를 나타낸 것입니다.  
[그림] * 해당 그림은 Sequence Diagram을 Text로 전환하였습니다.  
launchPurchaseFlow()
---
1. App -> SDK : startConnection()
2. SDK -> ONE store : bindService()
3. ONE store -> SDK : onServiceConnected()
4. SDK -> ONE store : isBillingSupportedExtParams()
5. ONE store -> SDK : return iapResult
6. SDK -> APP : onSetupFinished()
7. APP -> ONE store : queryProductDetailAsync()
8. ONE store -> App : return List<ProductDetail> (ProductDetailsLister receive data)
9. App -> SDK : launchPurchaseFlow()
10. SDK -> ONE store : getPurchaseInternalExtraParams()
11. ONE store -> SDK : return InApp Intent
12. SDK -> ONE store : start P

In [83]:
def extract_package(content: str) -> str:
    """Kotlin 파일에서 패키지 선언을 추출합니다."""
    package_match = re.search(r'package\s+([\w.]+)', content)
    return package_match.group(1) if package_match else ""

def extract_class_blocks(content: str) -> List[tuple]:
    """Kotlin 파일에서 클래스 블록을 추출합니다."""
    blocks = []
    
    # 클래스, 객체, 인터페이스, 함수 등 추출
    patterns = [
        (r'(class\s+\w+[\s\S]*?})', 'class'),
        (r'(object\s+\w+[\s\S]*?})', 'object'),
        (r'(interface\s+\w+[\s\S]*?})', 'interface'),
        (r'(fun\s+\w+[\s\S]*?})', 'function'),
        (r'(companion\s+object[\s\S]*?})', 'companion_object')
    ]
    
    for pattern, symbol_type in patterns:
        matches = re.finditer(pattern, content, re.MULTILINE)
        for match in matches:
            block_text = match.group(1)
            # 클래스/함수 이름 추출
            name_match = re.search(r'(class|object|interface|fun|companion\s+object)\s+(\w+)', block_text)
            symbol_name = name_match.group(2) if name_match else f"{symbol_type}_{len(blocks)}"
            
            blocks.append((f"{symbol_type}:{symbol_name}", block_text))
    
    return blocks

def load_kotlin_documents(project_root: str) -> list[Document]:
    """Kotlin 프로젝트에서 문서를 로드합니다."""
    project_root_path = Path(project_root)
    documents = []

    for file_path in project_root_path.rglob("*.kt"):
        rel_path = file_path.relative_to(project_root_path)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()

        package = extract_package(content)
        blocks = extract_class_blocks(content)
        
        for symbol, block in blocks:
            doc = Document(
                page_content=block,
                metadata={
                    "source": str(rel_path),
                    "symbol": symbol,
                    "package": package,
                    "type": "code",
                    "language": "kotlin"
                }
            )
            documents.append(doc)

    return documents

# Kotlin 코드 로드
docs_kotlin = load_kotlin_documents("code_sample/onestore_iap_release/")
print(f"Kotlin 코드 문서 로드 완료: {len(docs_kotlin)}개 청크")

# 샘플 출력
if docs_kotlin:
    print("\n첫 번째 코드 문서 샘플:")
    print(f"메타데이터: {docs_kotlin[0].metadata}")
    print(f"내용 (처음 200자): {docs_kotlin[0].page_content[:200]}...")

Kotlin 코드 문서 로드 완료: 218개 청크

첫 번째 코드 문서 샘플:
메타데이터: {'source': 'onestore_iap_sample/sample_subscription/src/main/java/com/onestore/sample/subscription/base/BaseRecyclerViewAdapter.kt', 'symbol': 'class:BaseRecyclerViewAdapter', 'package': 'com.onestore.sample.subscription.base', 'type': 'code', 'language': 'kotlin'}
내용 (처음 200자): class BaseRecyclerViewAdapter<T>: RecyclerView.Adapter<RecyclerView.ViewHolder>() {
    interface OnItemClickListener {
        fun <T> onItemClick(position: Int, item: T)
    }...


In [84]:
def embed_and_save(docs: List[Document], output_path: str):
    """문서를 임베딩하고 FAISS 데이터베이스로 저장합니다."""
    # 임베딩 모델 초기화
    embedding_model = OllamaEmbeddings(model="deepseek-coder:6.7b")
    
    # FAISS 데이터베이스 생성 및 저장
    db = FAISS.from_documents(docs, embedding_model)
    db.save_local(output_path)
    print(f"✅ 임베딩 저장 완료: {output_path}")

# 모든 문서 통합
total_docs = docs_markdown + docs_kotlin
print(f"총 {len(total_docs)}개의 문서를 생성하였습니다.")
print(f"- 마크다운 문서: {len(docs_markdown)}개")
print(f"- Kotlin 코드: {len(docs_kotlin)}개")

# 임베딩 생성 및 저장
output_dir = "models/faiss_vs_rag_iap_v11_deepseek"
os.makedirs(output_dir, exist_ok=True)
embed_and_save(total_docs, output_dir)

총 337개의 문서를 생성하였습니다.
- 마크다운 문서: 119개
- Kotlin 코드: 218개
✅ 임베딩 저장 완료: models/faiss_vs_rag_iap_v11_deepseek


In [94]:
# output_dir = "models/faiss_vs_rag_iap_v7_home"
# 저장된 임베딩 로드 및 테스트
embedding_model = OllamaEmbeddings(model="deepseek-coder:6.7b")
loaded_db = FAISS.load_local(
    folder_path= output_dir, ##"models/faiss_vs_rag_iap_v10_home",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True,
)

# 검색 테스트
test_queries = [
    # "인앱결제 초기화 방법",
    # "정기결제 구현 코드",
    # "PNS 설정 방법",
    # "결제 검증 API",
    "launchPurchaseFlow()"
]

for query in test_queries:
    print(f"\n🔍 검색 쿼리: {query}")
    results = loaded_db.similarity_search(query, k=10)
    
    for i, doc in enumerate(results, 1):
        print(f"\n결과 {i}:")
        print(f"타입: {doc.metadata.get('type', 'unknown')}")
        print(f"소스: {doc.metadata.get('source', 'unknown')}")
        print(f"내용: {doc.page_content}")
        print("-" * 50)


🔍 검색 쿼리: launchPurchaseFlow()

결과 1:
타입: code
소스: onestore_iap_sample/sample_subscription/src/main/java/com/onestore/sample/subscription/ui/MainActivity.kt
내용: fun onTabUnselected(tab: TabLayout.Tab?) {}
--------------------------------------------------

결과 2:
타입: code
소스: onestore_iap_sample/sample_subscription/src/main/java/com/onestore/sample/subscription/ui/MainActivity.kt
내용: fun onTabReselected(tab: TabLayout.Tab?) {}
--------------------------------------------------

결과 3:
타입: documentation
소스: dev_center_guide_touched.md
내용: [ > 01. 원스토어 인앱결제 개요 > 결제 프로세스 > 인앱 상품 구매 요청하기]

상품을 구매하려면 먼저 상품 상세정보(queryProductDetailAsync) API 호출하여 상품의 정보을 가지고 launchPurchaseFlow API를 통해 구매 화면을 호출합니다.
결제가 완료되면 PurchaseClient 객체를 초기화할 때 입력한 PurchasesUpdatedListener를 통해 구매정보를 전달받을 수 있습니다.
결과 코드값을 검증한 후에 성공으로 확인되면 인앱 상품에 해당하는 아이템을 지급하는 프로세스를 진행합니다.
아래 그림은 개발사 앱과 원스토어 사이의 인앱 상품을 구매하는 프로세스를 나타낸 것입니다.  
[그림] * 해당 그림은 Sequence Diagram을 Text로 전환하였습니다.  
launchPurchaseFlow()
---
1. App -> SDK : startConnec

In [96]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# RAG 체인 구성 요소
embedding_model = OllamaEmbeddings(model="deepseek-coder:6.7b")
llm = ChatOllama(model="deepseek-coder:6.7b", temperature=0.3)

# 검색기 설정
retriever = loaded_db.as_retriever(
    search_type="mmr",
    # search_type="similarity",
    search_kwargs={"k": 10, "fetch_k": 40,"lambda_mult": 0.6}
)

docs = retriever.invoke("PNS는 무엇인가요? ")

for doc in docs:
    print(f"문서 제목: {doc.metadata.get('title', '제목 없음')}")
    print(f"문서 내용: {doc.page_content}...")  # 처음 200자만 출력
    print("-" * 40)  # 구분선

문서 제목: 원스토어 인앱 결제 연동 가이드
문서 내용: [ > 08. 정기 결제 적용하기 > 정기 결제 가격 변경하기 > 두 번의 가격 변경이 발생한 경우]

첫 번째 가격 변경 이후 두 번째 가격 변경이 7일 이내라면, 첫 번째 가격 변경은 무효화되고 두 번째 가격 변경만 적용이 됩니다.  
다만, 7일 이후라면 사용자는 두 번의 가격 변경에 대한 동의를 해야합니다.  
가격 변경에 대한 동의 절차는 사용자의 불편을 초래하며, 해당 정기 결제 상품을 유지하는 고객이 줄어들 가능성이 높아집니다.  
다시 한 번 강조하지만 가격 변경은 신중해야 합니다....
----------------------------------------
문서 제목: 원스토어 인앱 결제 연동 가이드
문서 내용: [ > 01. 원스토어 인앱결제 개요 > 결제 프로세스]

원스토어 인앱결제는 크게 다음의 네 가지 프로세스로 구성되어 있습니다....
----------------------------------------
문서 제목: 원스토어 인앱 결제 연동 가이드
문서 내용: [ > 02. 사전준비 > 원스토어 앱 설치하기]

개발자나 사용자가 원스토어 인앱결제를 이용하기 위해서는 원스토어 앱이 필요합니다.  
- [원스토어 앱 다운로드 안내](https://onestore-dev.gitbook.io/dev/policy/download)...
----------------------------------------
문서 제목: 원스토어 인앱 결제 연동 가이드
문서 내용: [ > 08. 정기 결제 적용하기 > 정기 결제 상품 변경 하기]

사용자는 정기 결제 이용 중에 좀 더 좋은 혹은 저렴한 정기 결제 상품으로 플랜을 변경하고 싶을 수 있습니다.  
사용자는 다른 정기 결제 상품을 결제하여 정기 결제 상품을 변경 할 수 있고, 개발자는 비례 배분 모드를 설정하여 정기 결제 상품 변경을 처리 할 수 있습니다.  
설정 할 수 있는 비례 배분 모드(PurchaseFlowParams.Prora

In [92]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# RAG 체인 구성 요소
embedding_model = OllamaEmbeddings(model="deepseek-coder:6.7b")
llm = ChatOllama(model="deepseek-coder:6.7b", temperature=0.3)

# 검색기 설정
retriever = loaded_db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 8, "fetch_k": 20, "lambda_mult": 0.6}
)

# 프롬프트 템플릿
prompt_template = """
당신은 원스토어 인앱결제 전문가입니다. 주어진 컨텍스트를 바탕으로 질문에 답변해주세요.

답변 시 다음 사항을 고려해주세요:
1. 한국어로 명확하고 이해하기 쉽게 답변하세요
2. 코드 예시가 있다면 포함해주세요
3. 단계별로 설명해주세요
4. 개발자 관점에서 실용적인 정보를 제공해주세요
5. 컨텍스트에 없는 내용은 "해당 정보를 찾을 수 없습니다"라고 답변하세요

컨텍스트:
{context}

질문: {question}

답변:"""

prompt = PromptTemplate.from_template(prompt_template)

# RAG 체인 구성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ RAG 체인이 생성되었습니다!")

✅ RAG 체인이 생성되었습니다!


In [93]:
# 테스트 질문들
test_questions = [
    # "원스토어 인앱결제 초기화는 어떻게 하나요?",
    # "정기결제 구현 방법을 알려주세요",
    # "PNS(Push Notification Service) 설정 방법은?",
    # "결제 검증은 어떻게 하나요?",
    # "관리형 상품과 구독형 상품의 차이점은?",
    "PNS는 무엇인가요?"
]

print("🤖 RAG 체인 테스트 시작\n")
print("=" * 80)

for i, question in enumerate(test_questions, 1):
    print(f"\n📝 질문 {i}: {question}")
    print("-" * 60)
    
    try:
        answer = rag_chain.invoke(question)
        print(answer)
    except Exception as e:
        print(f"❌ 오류 발생: {str(e)}")
    
    print("=" * 80)

🤖 RAG 체인 테스트 시작


📝 질문 1: PNS는 무엇인가요?
------------------------------------------------------------
PNS(Push Notification Service)는 푸시 알림을 전송하는 서비스를 말합니다. 각 플랫폼(iOS, Android)별로 PNS를 이용하여 푸시 알림을 전송할 수 있게 해주는 서비스를 말합니다. PNS를 이용하면, 사용자가 언제든지 서비스에 접속하지 않아도 시스템이 자동으로 사용자에게 알림을 보낼 수 있습니다.

PNS를 통해 제공되는 주요 기능은 다음과 같습니다:
1. 푸시 알림: 사용자에게 어떠한 상황이나 이벤트에 바로 알림을 보내는 기능
2. 토큰: PNS에 등록되면, 각 사용자마다 할당되는 고유의 토큰을 받을 수 있는 기능
3. 타겟팅: 사용자의 취향, 활동 정보 등을 기반으로 특정 사용자에게 푸시 알림을 보내는 기능
4. 통계: PNS는 푸시 알림의 전송, 수신 횟수 등의 통계를 제공

PNS는 메시지 브로드캐스팅, 타겟팅 메시징, 이벤트 알림 등 다양한 용도로 사용될 수 있습니다.



In [ ]:
#########

In [49]:
def chunk_based_markdown_split(md_text: str, chunk_size: int = 500, chunk_overlap: int = 100) -> list[Document]:
    """
    마크다운 문서를 chunk_size와 chunk_overlap 기반으로 분할합니다.
    
    Args:
        md_text: 마크다운 텍스트
        chunk_size: 청크 크기 (기본값: 500)
        chunk_overlap: 청크 간 겹치는 문자 수 (기본값: 100)
        
    Returns:
        list[Document]: 분할된 문서 리스트
    """
    from langchain_text_splitters import RecursiveCharacterTextSplitter
    
    # RecursiveCharacterTextSplitter 사용
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ". ", " ", ""],  # 마크다운에 적합한 구분자
        length_function=len,
    )
    
    docs = splitter.split_text(md_text)
    
    # Document 객체로 변환하고 메타데이터 추가
    result_docs = []
    for i, doc_text in enumerate(docs):
        # 헤더 정보 추출 (첫 번째 줄에서 # 확인)
        lines = doc_text.split('\n')
        header_info = ""
        content_start = 0
        
        for j, line in enumerate(lines):
            if line.strip().startswith('#'):
                header_info = line.strip()
                content_start = j + 1
                break
        
        # 헤더 정보를 포함한 내용 구성
        if header_info:
            content = f"[{header_info}]\n\n" + '\n'.join(lines[content_start:])
        else:
            content = doc_text
        
        doc = Document(
            page_content=content,
            metadata={
                "type": "documentation",
                "source": "dev_center_guide_touched.md",
                "chunk_idx": i,
                "chunk_size": chunk_size,
                "chunk_overlap": chunk_overlap,
                "header": header_info if header_info else "no_header"
            }
        )
        result_docs.append(doc)
    
    return result_docs

# 테스트: chunk_based_markdown_split 사용
print("=== Chunk 기반 분할 테스트 ===")
str_md_file = load_markdown_file("data/dev_center_guide_touched.md")
docs_chunk_based = chunk_based_markdown_split(str_md_file, chunk_size=500, chunk_overlap=100)
print(f"Chunk 기반 분할 완료: {len(docs_chunk_based)}개 청크")

# launchPurchaseFlow 포함하는 문서 확인
print("\n=== launchPurchaseFlow 포함 문서 확인 ===")
launch_docs = []
for i, doc in enumerate(docs_chunk_based):
    if 'launchPurchaseFlow' in doc.page_content:
        launch_docs.append((i, doc))
        print(f"문서 {i}: {doc.page_content[:150]}...")
        print("-" * 50)

print(f"\n총 {len(launch_docs)}개의 문서에서 'launchPurchaseFlow' 발견")

# 새로운 임베딩 생성 (chunk 기반)
print("\n=== 새로운 임베딩 생성 ===")
total_docs_new = docs_chunk_based + docs_kotlin
print(f"새로운 총 문서 수: {len(total_docs_new)}개")
print(f"- Chunk 기반 마크다운: {len(docs_chunk_based)}개")
print(f"- Kotlin 코드: {len(docs_kotlin)}개")

# 새로운 임베딩 저장
output_dir_new = "models/faiss_vs_rag_iap_v10_chunk"
os.makedirs(output_dir_new, exist_ok=True)
embed_and_save(total_docs_new, output_dir_new)

=== Chunk 기반 분할 테스트 ===
Chunk 기반 분할 완료: 312개 청크

=== launchPurchaseFlow 포함 문서 확인 ===
문서 6: [##### 인앱 상품 구매 요청하기]

상품을 구매하려면 먼저 상품 상세정보(queryProductDetailAsync) API 호출하여 상품의 정보을 가지고 launchPurchaseFlow API를 통해 구매 화면을 호출합니다.
결제가 완료되면 PurchaseCl...
--------------------------------------------------
문서 7: launchPurchaseFlow()
---
1. App -> SDK : startConnection()
2. SDK -> ONE store : bindService()
3. ONE store -> SDK : onServiceConnected()
4. SDK -> ON...
--------------------------------------------------
문서 8: 9. App -> SDK : launchPurchaseFlow()
10. SDK -> ONE store : getPurchaseInternalExtraParams()
11. ONE store -> SDK : return InApp Intent
12. SDK -> ONE...
--------------------------------------------------
문서 50: [##### 구매 요청하기]

앱에서 구매 요청을 하기 위해서는 기본 스레드에서 launchPurchaseFlow() 함수를 호출합니다.

이 함수는 queryProductDetailsAsync() 함수를 호출해서 얻은 ProductDetail 객체의 값을 토대로 Pu...
--------------------------------------------------
문서 54: purchaseClient.launchPurchaseFlow(activity, purchaseFlowParams)

In [51]:
# output_dir1 = "models/faiss_vs_rag_iap_v7_home"
# 저장된 임베딩 로드 및 테스트
embedding_model = OllamaEmbeddings(model="exaone3.5:latest")
loaded_db1 = FAISS.load_local(
    folder_path=output_dir_new,
    embeddings=embedding_model,
    allow_dangerous_deserialization=True,
)

# 검색 테스트
test_queries = [
    # "인앱결제 초기화 방법",
    # "정기결제 구현 코드",
    # "PNS 설정 방법",
    # "결제 검증 API",
    "launchPurchaseFlow()"
]

for query in test_queries:
    print(f"\n🔍 검색 쿼리: {query}")
    results = loaded_db1.similarity_search(query, k=10)
    
    for i, doc in enumerate(results, 1):
        print(f"\n결과 {i}:")
        print(f"타입: {doc.metadata.get('type', 'unknown')}")
        print(f"소스: {doc.metadata.get('source', 'unknown')}")
        print(f"내용: {doc.page_content}")
        print("-" * 50)


🔍 검색 쿼리: launchPurchaseFlow()

결과 1:
타입: documentation
소스: dev_center_guide_touched.md
내용: launchPurchaseFlow()
---
1. App -> SDK : startConnection()
2. SDK -> ONE store : bindService()
3. ONE store -> SDK : onServiceConnected()
4. SDK -> ONE store : isBillingSupportedExtParams()
5. ONE store -> SDK : return iapResult
6. SDK -> APP : onSetupFinished()
7. APP -> ONE store : queryProductDetailAsync()
8. ONE store -> App : return List<ProductDetail> (ProductDetailsLister receive data)
9. App -> SDK : launchPurchaseFlow()
10. SDK -> ONE store : getPurchaseInternalExtraParams()
--------------------------------------------------

결과 2:
타입: code
소스: onestore_iap_sample/sample_subscription/src/main/java/com/onestore/sample/subscription/ui/MainActivity.kt
내용: fun onTabReselected(tab: TabLayout.Tab?) {}
--------------------------------------------------

결과 3:
타입: code
소스: onestore_iap_sample/sample_subscription/src/main/java/com/onestore/sample/subscription/ui/MainActivity.kt
내용: fun onTabUnse

In [59]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# RAG 체인 구성 요소
embedding_model = OllamaEmbeddings(model="exaone3.5:latest")
llm = ChatOllama(model="exaone3.5:latest", temperature=0.3)

# 검색기 설정
retriever = loaded_db1.as_retriever(
    search_type="mmr",
    # search_type="similarity",
    search_kwargs={"k": 10, "fetch_k": 20, "lambda_mult": 0.6}
)

docs = retriever.invoke("launchPurchaseFlow()는 무엇인가요?")

for doc in docs:
    print(f"문서 제목: {doc.metadata.get('title', '제목 없음')}")
    print(f"문서 내용: {doc.page_content[:200]}...")  # 처음 200자만 출력
    print("-" * 40)  # 구분선

문서 제목: 제목 없음
문서 내용: [##### Constants]

- ALL
String ALL
상품상세 정보를 한번에 받기 위한 타입
Constant Value: "all"

- INAPP
String INAPP
관리형 상품 타입
Constant Value: "inapp"

- AUTO
String AUTO
월정액 상품 타입
Constant Value: "auto"

- SUBS
Str...
----------------------------------------
문서 제목: 제목 없음
문서 내용: PurchaseFlowParams purchaseFlowParams = PurchaseFlowParams.newBuilder()
      .setProductId(productId)
      .setProductType(productType)
      .setDeveloperPayload(devPayload)    // optional
      .s...
----------------------------------------
문서 제목: 제목 없음
문서 내용: [##### 구매 처리하기]

구매가 완료되면 앱에서 구매 확인 처리를 해야 합니다. 대부분의 경우 앱은 PurchasesUpdatedListener를 통해 구매 알림을 받습니다.
또는  구매 내역 조회하기에서 설명된 것처럼 앱이 PurchaseClient.queryPurchasesAsync() 함수를 호출하여 처리하는 경우가 있습니다.

다음 메서드 중 ...
----------------------------------------
문서 제목: 제목 없음
문서 내용: [#### 정기 결제 관리 화면 열기]

원스토어는 사용자가 정기 결제 상품을 쉽게 관리하도록 정기 결제 관리 메뉴를 제공합니다.  

매개변수로 SubscriptionsParams에 PurchaseData를 포함해서 넣으면 구매 데이터를 확인하여 해당 정기 결제 상품의 관리 화면을 실행합니다.
SubscriptionParams을

In [55]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# RAG 체인 구성 요소
embedding_model = OllamaEmbeddings(model="exaone3.5:latest")
llm = ChatOllama(model="exaone3.5:latest", temperature=0.3)

# 검색기 설정
retriever1 = loaded_db1.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 8, "fetch_k": 20, "lambda_mult": 0.6}
)

# 프롬프트 템플릿
prompt_template = """
당신은 원스토어 인앱결제 전문가입니다. 주어진 컨텍스트를 바탕으로 질문에 답변해주세요.

답변 시 다음 사항을 고려해주세요:
1. 한국어로 명확하고 이해하기 쉽게 답변하세요
2. 코드 예시가 있다면 포함해주세요
3. 단계별로 설명해주세요
4. 개발자 관점에서 실용적인 정보를 제공해주세요
5. 컨텍스트에 없는 내용은 "해당 정보를 찾을 수 없습니다"라고 답변하세요

컨텍스트:
{context}

질문: {question}

답변:"""

prompt = PromptTemplate.from_template(prompt_template)

# RAG 체인 구성
rag_chain1 = (
    {"context": retriever1, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ RAG 체인이 생성되었습니다!")

✅ RAG 체인이 생성되었습니다!


In [58]:
# 테스트 질문들
test_questions = [
    # "원스토어 인앱결제 초기화는 어떻게 하나요?",
    # "정기결제 구현 방법을 알려주세요",
    "PNS는 무엇인가요?",
    # "결제 검증은 어떻게 하나요?",
    # "관리형 상품과 구독형 상품의 차이점은?",
    "launchPurchaseFlow()에 대해서 알려주세요."
]

print("🤖 RAG 체인 테스트 시작\n")
print("=" * 80)

for i, question in enumerate(test_questions, 1):
    print(f"\n📝 질문 {i}: {question}")
    print("-" * 60)
    
    try:
        answer = rag_chain1.invoke(question)
        print(answer)
    except Exception as e:
        print(f"❌ 오류 발생: {str(e)}")
    
    print("=" * 80)

🤖 RAG 체인 테스트 시작


📝 질문 1: PNS는 무엇인가요?
------------------------------------------------------------
제공된 컨텍스트에서 **PNS**에 대한 직접적인 정의나 설명은 포함되어 있지 않습니다. 따라서 해당 정보를 찾을 수 없습니다.

만약 **PNS**가 특정 컨텍스트 내에서 사용되는 약어나 용어라면, 추가적인 맥락이나 정보가 필요합니다. 일반적으로 인앱 결제나 원스토어 관련 문서에서 자주 사용되는 용어가 아니라면, 개발자 커뮤니티나 공식 문서에서 더 자세한 정의를 찾아볼 것을 권장드립니다.

만약 다른 정보나 맥락이 필요하시다면, 좀 더 구체적인 정보를 제공해 주시면 도움을 드릴 수 있을 것 같습니다.

📝 질문 2: launchPurchaseFlow()에 대해서 알려주세요.
------------------------------------------------------------
`launchPurchaseFlow()` 메서드는 원스토어 인앱 결제 시스템에서 사용자가 인앱 상품을 구매하는 프로세스를 시작하는 핵심 함수입니다. 개발자 관점에서 이 메서드를 사용하는 방법과 주요 단계를 아래에 설명드리겠습니다.

### 주요 기능 및 사용법

1. **목적**:
   - 사용자가 인앱 상품을 구매하도록 유도하고, 구매 프로세스를 처리합니다.
   - 구매 완료 후 앱 측에서 구매 확인 로직을 실행할 수 있게 합니다.

2. **매개변수**:
   - `activity`: 구매 프로세스를 진행할 액티비티 객체입니다.
   - `purchaseFlowParams`: 구매 플로우 설정을 위한 `PurchaseFlowParams` 객체입니다. 이 객체를 통해 상품 정보, 타입, 개발자 페이로드 등을 설정할 수 있습니다.

### 단계별 설명 및 코드 예시

#### 1. `PurchaseFlowParams` 객체 생성
`PurchaseFlowParams` 객체를 생성하고 필요한 파라미터를 설정합